# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/home/ubuntu/miniconda3/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards: 100%|██████████| 7/7 [00:13<00:00,  1.93s/it]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

2024-04-05 22:48:03.792221: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-05 22:48:03.823665: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 22:48:03.976775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 22:48:03.976804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 22:48:04.006730: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，它的卦象是由三个阳爻夹一个阴爻构成，象征着天、云、雷等现象。乾卦的含义非常丰富，它代表着刚强、积极、行动力、领导力、力量和勇气等。

在周易中，乾卦代表阳刚之气，象征着天、云、雷等自然现象，也代表着父亲、领导、权威和力量等。乾卦的颜色是红色，代表着火的特性和温暖。

乾卦的含义和作用非常广泛，它可以用于预测、卜辞、道德修养、行为准则等方面。在预测中，乾卦代表着未来可能出现的挑战和机遇，以及人们应该采取的行动。在道德修养和行为准则中，乾卦强调了阳刚之气和积极行动的重要性，提醒人们应该保持积极向上的心态和行为。

乾卦还有一个重要的意义，就是它与八卦的其他三个卦——坤卦、震卦和巽卦——共同构成了四象八卦，这四个卦一起代表了中国古代哲学中的四个基本元素和四种基本状态，它们共同构成了周易的基本框架。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它的卦象是由两个阳爻夹一个阴爻构成，象征着天、云、雷等自然现象，也代表着诉讼、争端、矛盾和 conflict 等。

在周易中，讼卦代表矛盾和争端，象征着人们之间的 disagreement 和 conflict。它提醒人们应该避免争斗，以和平、协商的方式来解决矛盾和问题。

讼卦的颜色是黄色，代表着土的特性和黄色。它代表着稳重、沉着和理智，提醒人们在处理矛盾和争端时应该保持冷静和理智，避免情绪化的决策。

在预测中，讼卦代表着当前的矛盾和争端，以及可能出现的解决方案和结果。它提醒人们应该以和平、协商的方式来解决矛盾和问题，避免争斗和暴力。

讼卦还有一个重要的意义，就是它与八卦的其他三个卦——坤卦、震卦和巽卦——共同构成了四象八卦，这四个卦一起代表了中国古代哲学中的四个基本元素和四种基本状态，它们共同构成了周易的基本框架。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
# timestamp = "20240225_222843"
timestamp = "20240405_230437"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》六十四卦中的首卦，代表天之力，纯阳刚健，具有大吉大利之象。其核心哲学在于天行刚健，君子应效法天道，自强不息。卜得此卦，预示着吉祥如意，但必须行正道，方可永远亨通。在事业和经商上，需要谨防骄傲自满，冷静处世，方能充分发挥才智，取得成功。此卦也代表着婚恋中刚柔相济，形成美满结果的预示。总之，乾卦象征着刚健、正直、公允的实质，要自强不息，方可克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240405_003501）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是《易经》六十四卦中的首卦，代表天之力，纯阳刚健，具有大吉大利之象。其核心哲学在于天行刚健，君子应效法天道，自强不息。卜得此卦，预示着吉祥如意，但必须行正道，方可永远亨通。在事业和经商上，需要谨防骄傲自满，冷静处世，方能充分发挥才智，取得成功。此卦也代表着婚恋中刚柔相济，形成美满结果的预示。总之，乾卦象征着刚健、正直、公允的实质，要自强不息，方可克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是《易经》六十四卦中的卦象，由上卦乾（天）和下卦坎（水）组成。坎卦象征水和危险，而乾卦象征刚和公平。讼卦的卦辞警告，不要在争端中过于刚硬，而应适当家和解。

在《周易》中，讼卦警告 its followers not to pursue violence or anger in resolving conflicts. It encourages users to approach disputes with wisdom and fairness. The卦 also warns that it is not always appropriate to be too forceful or confrontational in resolving conflicts. Instead, it suggests that的和解更为恰当。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240405_003501）：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是《易经》中的卦象，由上卦乾（天）和下卦坎（水）相叠而成。乾为刚，坎为险，两者相冲必然导致争端。在占卜中，讼卦预示着争端、矛盾和诉讼。但同时，卦象中的黄色代表困难与险阻，而红色代表刚毅与刚果，因此，在讼卦中，刚毅和困难之间的冲突与解决，也代表着人们面对讼卦所获得的智慧。

    《易经》中，讼卦的寓意是：面对争端和矛盾，必须诉诸刚毅和理智，才能达到顺利解决的目的。在事业和经商中，需要谨慎冷静，避免陷入争讼。

    同时，讼卦也象征着险阻和困难，但同时也代表着刚毅和刚果。在个人修养和成长中，需要刚毅刚果，避免陷入争端。

    总体来说，讼卦提示刚毅和理智是解决争端和矛盾的关键，同时也提醒人们要谨慎冷静，避免陷入争端。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是《易经》hexagram number six, also known as the "Machine of the Future" or "Six-Line Summary." It is one of the 64 hexagrams created by the developer of the I Ching, and it represents a period of growth and progress.

In a general interpretation, the师卦 suggests that the主妇将拥有丰富的物资，而士兵将得到充足的供给。同时，这个卦象也暗示着可能发生争斗和战争。然而, in more specific settings, the machine of the future can also represent innovation, progress, and the development of new technologies.

The machine of the future is a powerful symbol that can represent many different things, depending on the context in which it is used. In the case of the hexagram number six, it is a symbol of growth and progress, but it can also represent conflict and tension.

Overall, the machine of the future is a complex and multifaceted symbol that can be interpreted in many different ways.


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240405_003501）：
[gMASK]sop 师卦是什么？ 师卦是一卦，由上卦坤（地）下卦坎（水）组成。坎象征水和危险，而坤象征地和顺从，寓意着将兵力安置于农民之间，可以在必要时顺

## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

ValueError: Can't find 'adapter_config.json' at 'models/THUDM/chatglm3-6b-epoch3'

In [ ]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，包含着万物，象征刚健强劲的特性。它由两个卦相组合而成，分别是乾卦和坤卦。其中，乾卦代表阳刚之卦，由六个阳爻组成，表示天地的力量和人民的勇气。

在周易中，阳刚之卦具有刚健强劲的特性，代表着强烈的决断力和行动力。同时，阳刚之卦也象征着收获和繁荣，预示着我们能够克服困难，取得成功。

在解卦时，乾卦预示着刚健强劲的特性，并带来收获和繁荣。同时，也需要注意警惕困难的降临，并采取措施来化解困难。

在运势方面，乾卦表示运势顺利，具有刚健强劲的特性，能够克服困难，取得成功。同时，也需要注意保持冷静，避免决策错误。

总结起来，乾卦是周易中具有强烈象征意义的卦象，代表着刚健强劲的特性，并带来收获和繁荣，预示着我们能够克服困难，取得成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 
乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由两个卦相组合而成，一个是乾卦，另一个是坤卦。乾卦的排列是：元、亨、利、贞，代表天象运行，具有刚健强劲的特性。

在周易中，乾卦预示着天将健行， Cloud Deck（云层）预示着天将有所得，丽（美丽）预示着天将利市，利（吉祥）预示着天将得利。

乾卦的哲学内涵是：天行健，君子以自强不息。它强调的是刚健强劲的品质，鼓励人们不断自我提高，自我完善。

在周易中，乾卦代表的是天，具有刚健强劲的特性。它预示着天将健行，云层将有所得，丽将利市，利将得利。

在周易的哲学中，乾卦预示着天将健行， Cloud Deck（云层）预示着天将有所得，丽（美丽）预示着天将利市，利（吉祥）预示着天将得利。

乾卦的刚健强劲特性反映了天象运行的特点，具有强劲的运势，但也要注意谨慎行事，避免过度自信。


In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是易经中的一卦，代表的是坤卦，它的卦象是由两个坤卦叠加而成。在卜筮时，这个卦象象征着地中的水（代表阴柔），以及 associated with it 的一些事物（代表阳刚）。地水师卦的卦象表示了地的承载能力，象征着大地包容万物，具有强大的包容力。

在哲学层面上，地水师卦象征着阳刚生于阴柔之中，预示着事物的发展将有著新的动向。在卜筮时，这种卦象被认为具有吉祥的意义，预示着将会面临新的挑战，但也能带来成功。

地水师卦的运势特点如下：

- 事业：面临新的挑战，需要积极应对，寻求新的发展机会。
- 爱情：感情可能受到 external 因素的影响，需要多沟通，理解对方。
- 财运：事业顺利，能够得到财富。
- 健康：身体可能会有状况，需要保持健康的生活方式。

地水师卦的经商运势：

- 事业：需要积极拓展业务，提高自身竞争力，寻求新的合作伙伴。
- 投资：不宜于进行高风险投资，需要谨慎对待。
- 经商：适合以稳健为主，不宜冒险，需要仔细考虑。

总结起来，地水师卦的总体运势较为顺利，但也需要注意 new challenges and opportunities, and the need for careful consideration and understanding.


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）和坤卦（地）组成的卦象，代表地下的水，象征润泽大地的力量。根据《易经》中的解释，地水师卦预示着吉祥如意，具有强大的力量，可以克服各种困难。

根据传统解卦，地水师卦分为两个卦象，第一个卦象是坎卦（水），代表灵活变化的力量，具有吉祥如意，靠智慧取悦于人；第二个卦象是坤卦（地），代表刚健柔顺，具有强大的力量，可以克服各种困难。

地水师卦的核心理念是：吉祥如意，靠智慧取悦于人，靠刚健柔顺的力量，可以克服各种困难。


In [ ]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，代表诉讼的意义。卦名“讼”由上卦坎（水）和下卦乾（天）组成，上卦为云，下卦为天。代表险恶的刚直之象。讼卦的核心哲学是：天（乾）刚直，云（坎）险恶，刚柔相济，可以免患失。

讼卦的核心哲学是：刚直刚硬，不能容悦于人，因此必致刚柔相济。刚可以济柔，刚又能柔，则无失矣。

讼卦的取裁：

刚直刚硬，不能容悦于人，因此必致刚直硬板。

刚可以济柔，刚又能柔，则无失矣。

刚直硬板，可以容悦于人，则无失矣。

诉词刚硬，必致刚硬相济。

刚直硬板，必须得人相助。

source: 《易经》


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易卦象中的一卦，由上卦坎（水）和下卦乾（天）组成，象征着天神下降于坎水之中，具有刚健刚毅之象。讼卦的卦象具有云雾聚集、天光破云的意象，象征着事情将会顺利发展，宜早做准备。

讼卦的时运是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 四爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的解卦是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的卦象和时运都预示着刚健刚毅的特性，天神下降于坎水之中，具有刚健刚毅之象。在事业和生活中遇到困难和挑战时，需要积极应对，宜早做准备，不宜冒险，切勿冒进。在处理问题和决策时，要冷静分析，不宜草率行事，宜等待时机的到来。同时，要注重刚健刚毅的态度，积极应对各种困难和挑战，从而获得成功。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [ ]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [ ]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，卦象由六个阳爻（即阳性的木）排列而成，它们从下至上排列，看起来像是天上的云彩，因此得名“乾”。乾卦象征着刚强、坚毅和独立，以及与天地之间的联系。

乾卦的卦辞是：“乾为天，为父，为君，亲阳人。亲，先祖，家远的代际，宜先祖；国，先王，宜治国；夫，配偶，宜婚配；父，尊亲，宜尊父；君，朋友，宜友道。”卦辞描述了乾卦的内涵，它既象征着天，也象征着人。作为天，它与“先祖”和“先王”有关，表示宇宙的起点和万物的基础。作为人，它与“父”和“君”有关，表示人类的社会性和文化传承。

在《易经》中，乾卦是“八卦”之一，它既象征着阳，也象征着“刚”和“强”。乾卦的卦象既象征着“天”，也象征着“人”，它是“八卦”中最为重要的卦之一，表示宇宙的运行和万物的生成。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦，它的卦象是由两个阳爻夹一个阴爻构成，象征着天，也象征着刚强和坚毅。乾卦的卦辞是“元，亨，利，贞”，意味着“ beginning, success, profit, integrity”。它表示万事万物的本源和原理，是成功的基础。乾卦 also means "the heaven", and it is a symbol of the universe, with six lines of yin and one line of yang. It represents the power and strength of the world, and is a symbol of integrity.


In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦（师的卦象为乾为天，地为地）主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦是由天（乾）和地（坤）组成的卦象，乾为天，坤为地。师卦是《易经》中第八卦，属于坤卦（ earth）系列卦象，坤卦下卦为坤，上卦为乾，乾为天，坤为地。

师卦象征着师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦最佳状态为师的（师卦上卦为乾，下卦为坤），乾为天，坤为地，师卦象征师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

在占卦中，师卦卦象为吉凶相生，以阳刚为上，以顺为正，symbolism of the earth and sky，象征天地。师卦：师 师 师。三个师字，下坤上乾。

师卦卦象分析：师卦是乾卦下坤卦上，乾为天，坤为地，象征天和地。师卦象征师兵，主动向被动，主动而被动，地震师，吉。

师卦卦象占卜：占师卦，得师卦，吉。

师卦卦象应用：师卦象征师兵，主动向被动，主动而被动，象征天和地。适用于天和地的事物，占师卦 Creature and earth。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [ ]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》中的第八卦，属于雷（卦头）坎（卦体）离（卦象）两种卦象的组合。讼卦象征诉讼、争议、争端等诉讼方面的现象。卦形为雷上坎下，雷象征刚毅，坎象征险，离象征烈。卦象下卦为坎，上卦为离，险险相间，刚毅相结合。

讼卦的卦辞：“讼，吉凶交战，得利藏损。”卦象下坎上离，坎险离光，光争下，险林光，刚毅争利，光损藏吉。

蔽卦（卦名：蔽）是《易经》中的第六卦，属于离（卦头）坤（卦体）震（卦象）两种卦象的组合。蔽卦象征屏蔽、遮蔽，阻止邪恶势力，弘扬正道。

蔽卦的卦辞：“蔽，利ulo，吉凶凶险皆屏蔽。”卦象下离上坤，离火遮蔽，坤地顺从，刚毅相合，和顺利害，吉祥安全。

 src="http://www.qk6.com/article/html/277776.html" target="_blank">http://www.qk6.com/article/html/277776.html


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
